# Evaluation Technique Jimini

## Présentation de Jimini

Pour rappel, nous développons trois outils : 
- Un outil d'analyse factuelle de documents;
- Un outil de recherche juridique;
- Un outil de rédaction de documents juridiques, type contrats.

## Information Retrieval

### Objectif

Les deux premiers outils nécessitent d'être capable de récupérer le contexte pertinent dans un ensemble de paragraphes donné - problématique connue sous le nom d'`information retrieval` ou de `document retrieval`. L'objectif de cette partie est donc de construire un moteur de recherche juridique. Nous disposons d'un corpus de documents juridiques, et nous souhaitons que l'utilisateur puisse poser une question en langage naturel, puis que le moteur de recherche lui renvoie les documents les plus pertinents.

### Dataset 

Dans le cadre de ce notebook, nous nous intéressons à un jeu de données juridique francophone : [Belgian Statuatory Article Retrieval Dataset (BSARD)](https://huggingface.co/datasets/maastrichtlawtech/bsard), qui contient ~1000 questions posées par les utilisateurs d'une plateforme d'information juridique belge (Droits Quotidiens), ainsi que les articles pertinents pour y répondre. Ce jeu de données est issu de l'article [A Statutory Article Retrieval Dataset in French](https://arxiv.org/pdf/2108.11792.pdf) de l'Université de Maastricht.

Ainsi que détaillé sur la page HuggingFace, le dataset se présente sous la forme de deux fichiers CSV contenant les questions d'entraînement `questions_fr_train.csv` et de test `questions_fr_test.csv`, et structurés comme suit :
 - `id` : un attribut int32 correspondant à un numéro d'identifiant unique pour la question.
 - `question` : un attribut de type chaîne de caractères correspondant à la question.
 - `category` : un attribut de type chaîne de caractères correspondant au sujet général de la question.
 - `subcategory` : un attribut de type chaîne de caractères correspondant au sous-sujet de la question.
 - `extra_description` : un attribut de type chaîne de caractères correspondant aux tags de catégorisation supplémentaires de la question.
 - `article_ids` : un attribut de type chaîne de caractères contenant les ID d'articles pertinents pour la question, séparés par des virgules.

(À noter, ce dataset contient également un échantillon de ~113k questions synthétiques, ainsi que des exemples de *négatifs* durs, c'est-à-dire des questions avec des articles non-pertinents, dont vous pouvez librement vous servir pour entraîner votre modèle.)

Et d'un fichier `articles.csv` :
 - `id` : un attribut int32 correspondant à un numéro d'identifiant unique pour l'article.
 - `article` : un attribut de type chaîne de caractères correspondant à l'intégralité de l'article.
 - `code` : un attribut de type chaîne de caractères correspondant au code de loi auquel appartient l'article.
 - `article_no` : un attribut de type chaîne de caractères correspondant au numéro de l'article dans le code.
 - `description` : un attribut de type chaîne de caractères correspondant aux titres concaténés de l'article.
 - `law_type` : un attribut de type chaîne de caractères dont la valeur est soit "regional" soit "national".

### Évaluation

Ainsi que dans l'[article](https://arxiv.org/pdf/2108.11792.pdf), nous évaluerons la performance du modèle en utilisant les métriques R@100, R@200, R@500, MAP@100 et MRR@100.

Cependant, en conditions réelles, nous souhaitons que le moteur de recherche renvoie les documents les plus pertinents en premier. Nous utiliserons donc la métrique NDCG@100, qui prend en compte l'ordre des documents renvoyés, ainsi que les métriques précédentes, avec un $k$ plus petit (par exemple $k=1$, $k=5$ et $k=10$).

#### Pyterrier

On se sert ici du framework Pyterrier qui permet assez simplement de prototyper et évaluer un moteur de recherche, mais **libre au candidat d'utiliser l'approche qu'il préfère**.
Dans le code suivant, on se sert des notations de `PyTerrier`, qui fonctionne avec les clés suivantes :
- `qid` : `str` l'identifiant de la question
- `docno` : `str` l'identifiant de l'article
- `query` : `str` le texte de la question
- `rel_docnos` : `List[str]` la liste des documents pertinents pour la question
- `score`: `float` le score de pertinence du document pour la question
- `label`: `int` la pertinence du document pour la question (1 si pertinent, 0 sinon)
- `qrels` : `Dict[str, Dict[str, int]]` un dictionnaire contenant les documents pertinents pour chaque question


In [1]:
!pip install datasets==2.15.0 --quiet
!pip install pandas==2.1.3 --quiet
!pip install python-terrier==0.10.0 --quiet


In [2]:
# Retrieve the corpus of articles from the dataset
!wget https://huggingface.co/datasets/maastrichtlawtech/bsard/resolve/main/articles.csv --quiet


In [6]:
import os
import ast
import shutil
import pandas as pd
from datasets import load_dataset
import pyterrier as pt
from pyterrier.measures import *

if not pt.started():
    pt.init()


In [7]:
# Load the dataset
datasets = load_dataset("maastrichtlawtech/bsard")
train, test = datasets['train'], datasets['test']

# Convert to pandas dataframes
df_train = pd.DataFrame(train)
df_test = pd.DataFrame(test)
df_articles = pd.read_csv('articles.csv')

# Ensure that the article IDs are strings, and rename the column to 'docno', because of PyTerrier's expectations
df_articles['docno'] = df_articles['id'].astype(str)
df_articles["article"] = df_articles["article"].str.replace("[^a-zA-Z0-9 À-ÿ]", " ", regex=True).str.strip().str.replace(" +", " ", regex=True)

# Define a function to prepare the query dataframe
def prepare_queries(df):
    queries = pd.DataFrame({
        'qid': df['id'].astype(str),
        'query': df['question'].str.replace("[^a-zA-Z0-9 À-ÿ]", " ", regex=True).str.strip().str.replace(" +", " ", regex=True),
        'rel_docnos': [str(_id) for _id in df['article_ids']]
    })
    return queries

# Prepare the train and test query dataframes
train_queries = prepare_queries(df_train)
test_queries = prepare_queries(df_test)

# Convert relevance judgements to a DataFrame, for PyTerrier
qrels = pd.DataFrame([
    {'qid': str(qid), 'docno': str(docno), 'label': 1}
    for qid, docnos in test_queries[['qid', 'rel_docnos']].itertuples(index=False)
    for docno in ast.literal_eval(docnos)  # Convert the string representation of list back to a list
])


On indexe les articles dans l'index PyTerrier (`DFIndex` avait un bug, donc on passe par un `IterDictIndex`)

In [ ]:
# Indexing the articles using IterDictIndexer
if os.path.exists("./bsard_index"):
    shutil.rmtree("./bsard_index")
indexer = pt.IterDictIndexer("./bsard_index", overwrite=True)
docs = [{"docno": str(row["id"]), "text": row["article"]} for _, row in df_articles.iterrows()]
indexref = indexer.index(docs, fields=["text"])

index = pt.IndexFactory.of(indexref)


Le `retriever` doit être capable, étant donné une question, de classer les articles par pertinence. Le format de sortie est un CSV avec trois colonnes :
- `qid` : `str` l'identifiant de la question 
- `docno` : `str` l'identifiant de l'article
- `score` : `float` le score de pertinence de l'article pour la question

Libre au candidat de choisir l'approche qu'il préfère pour construire le `retriever`, l'essentiel étant qu'il construise un fichier CSV avec les colonnes `qid`, `docno` et `score`.

**Remarque** : Il peut ainsi y avoir plusieurs lignes pour une même question, si plusieurs articles sont pertinents (ce qui est souvent le cas)

In [9]:
# Define the retrieval model : for example dummy BM25 and TF-IDF
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF", properties={"c": 1.0})
bm25 = pt.BatchRetrieve(index, wmodel="BM25",  properties={"c": 1.0, "bm25.k_1": 1, "bm25.b": 0.6})

# Run the TF-IDF model and save the results
tfidf_results = tfidf.transform(test_queries)
tfid_results = tfidf_results[["qid", "docno", "score"]]
tfidf_results.to_csv("tfidf_results.csv", index=False)

# Run the BM25 model and save the results
bm25_results = bm25.transform(test_queries)
bm25_results = bm25_results[["qid", "docno", "score"]]  
bm25_results.to_csv("bm25_results.csv", index=False)


In [15]:
# Define the evaluation metrics
eval_metrics = ["recall_1", "recall_5", "recall_10", RR@10, AP@10, "recall_100", "recall_200", "recall_500", "map", "ndcg", RR@100, nDCG@100, AP@100]

# Load the results
tfidf_results = pd.read_csv("tfidf_results.csv")
bm25_results = pd.read_csv("bm25_results.csv")

# Evaluate the models
result = pt.Experiment(
    [tfidf_results, bm25_results],
    test_queries,
    qrels,
    eval_metrics,
    names=['TF-IDF', "BM25"],
)

print(result)


     name  recall_1  recall_5  recall_10     RR@10     AP@10  recall_100   
0  TF-IDF  0.062664  0.149757   0.208388  0.189779  0.108045    0.462643  \
1    BM25  0.047595  0.088109   0.119498  0.131880  0.069838    0.350239   

   recall_200  recall_500       map      ndcg    RR@100  nDCG@100    AP@100  
0    0.523497    0.583612  0.127457  0.254490  0.201457  0.221742  0.125681  
1    0.443217    0.510365  0.086245  0.200741  0.143892  0.159139  0.083980  


## Quelques questions ouvertes

### Information Retrieval

Dans le cas de *Jimini Analyzer*, on est dans un contexte un peu particulier, où l'information provient de documents structurés.
- L'approche précédente ne tient compte que du contenu des paragraphes, pas de leur place dans la structure du document. Comment pourrait-on améliorer cela ?
- Que pensez-vous du dataset utilisé ? Comment pourrait-on l'améliorer ?
- Quelle stratégie adopter pour faire un résumé d'un long document ?


### Raisonnement juridique


Récupérer le contexte pertinent est une étape *sine qua none*, mais il faut ensuite fournir le contexte au LLM, et qu'il réponde à la question posée. 
- Comment évaluer le LLM sur le contenu de ses réponses ? 
- Quelles métriques utiliser ? 
- Quel dataset utiliser / construire pour faire acquérir un raisonnement juridique au LLM ?
- Comment s'assurer que le dataset est de qualité / représentatif ?  
- De quel modèle partir ? 
- Quelle technique d'entraînement utiliser ? (full-parameter fine-tuning, PEFT - LoRA - QLoRA , prompt-tuning,...)
- Comment contrôler l'hallucination ?
- Comment faire de l'amélioration continue, en s'assurant que le modèle ne se dégrade pas lorsqu'on lui enseigne une nouvelle compétence ?
- Comment limiter les frais de calculs ?


### Rédaction de documents

- Comment rédiger un contrat de plusieurs dizaines de pages ?

### Bonus

Sentez-vous libre de proposer des améliorations, des idées, des pistes, etc. !


